In [1]:
# if run on colab
!pip install torcheeg
#from google.colab import mount
#drive.mount('/content/drive', force_remount=True) 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.5/214.5 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.8/230.8 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.7 MB/s eta 0:00:00
  Created wheel for torcheeg: filename=torc

In [2]:
import torcheeg
from torcheeg import transforms
from torcheeg.datasets import BCICIV2aDataset
from torcheeg.model_selection import KFoldGroupbyTrial
from torch.utils.data import DataLoader
from torcheeg.models import ATCNet, EEGNet
import torch
from torcheeg.trainers import ClassifierTrainer
import pytorch_lightning as pl

In [3]:
root_data_path = '/kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat'

In [4]:
dataset = BCICIV2aDataset(
    root_path=root_data_path,
    io_path=f'./examples_pipeline/bciciv-2a',
    # skip_trial_with_artifacts=True,
    # offline_transform=transforms.Compose([
    #     transforms.BandDifferentialEntropy(apply_to_baseline=True),
    #     transforms.To2d(apply_to_baseline=True),
    #     transforms.ToTensor(apply_to_baseline=True)
    # ]),
    online_transform=transforms.Compose([
        # transforms.To2d(apply_to_baseline=True),
        # transforms.ToTensor(apply_to_baseline=True),
        transforms.To2d(),
        transforms.ToTensor(),
        # transforms.CWTSpectrum(apply_to_baseline=True),
        # transforms.BandDifferentialEntropy(apply_to_baseline=True),
        # transforms.BaselineRemoval(),
    ]),
    label_transform=transforms.Compose([
        transforms.Select('label'),
        transforms.Lambda(lambda x: x - 1)
    ]),
    chunk_size=7*250,
    num_worker=2
)

[2024-05-16 04:52:07] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to ./examples_pipeline/bciciv-2a.
[2024-05-16 04:52:07] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]:   0%|          | 0/18 [00:00<?, ?it/s]
[RECORD /kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat/A04T.mat]: 0it [00:00, ?it/s]
[RECORD /kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat/A04T.mat]: 1it [00:01,  1.19s/it]
[RECORD /kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat/A04T.mat]: 13it [00:01, 13.55it/s]
[RECORD /kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat/A04T.mat]: 25it [00:01, 27.27it/s]
[RECORD /kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat/A04T.mat]: 35it [00:01, 38.24it/s]
[RECORD /kaggle/input/bci-competition

In [5]:
print("Dataset's info: ")
print(dataset.info)

Dataset's info: 
      start_at  end_at   clip_id subject_id  trial_id session subject  run  \
0          251    2001    A04T_0        A04         0       T     A04    1   
1         2254    4004    A04T_1        A04         1       T     A04    1   
2         4172    5922    A04T_2        A04         2       T     A04    1   
3         6124    7874    A04T_3        A04         3       T     A04    1   
4         8132    9882    A04T_4        A04         4       T     A04    1   
...        ...     ...       ...        ...       ...     ...     ...  ...   
5179     86751   88501  A04E_283        A04        43       E     A04    8   
5180     88657   90407  A04E_284        A04        44       E     A04    8   
5181     90585   92335  A04E_285        A04        45       E     A04    8   
5182     92699   94449  A04E_286        A04        46       E     A04    8   
5183     94758   96508  A04E_287        A04        47       E     A04    8   

      label  _record_id  
0         4   _recor

In [6]:
k_fold = KFoldGroupbyTrial(
    n_splits=10,
    split_path='./examples_pipeline/split',
    shuffle=True,
    random_state=44
)

In [7]:
DEVICE = 'gpu' if torch.cuda.is_available() else 'cpu'


for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=4
    )
    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=4
    )

    model = EEGNet(
        chunk_size=7*250,
        num_electrodes=22,
        num_classes=4
    )
    
    trainer = ClassifierTrainer(
        model=model,
        num_classes=4,
        lr=1e-4,
        weight_decay=1e-4,
        accelerator=DEVICE
    )

    trainer.fit(
        train_loader,
        val_loader, 
        max_epochs=50,
        default_root_dir=f'./examples_pipeline/eegnet_model/{i}',
        callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
        enable_progress_bar=True,
        enable_model_summary=True,
        limit_val_batches=0.0
    )

    score = trainer.test(
        val_loader,
        enable_progress_bar=True,
        enable_model_summary=True
    )[0]
    print(f"Fold {i} test accuracy: {score['test_accuracy']: .4f}")

[2024-05-16 04:52:53] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2024-05-16 04:52:53] INFO (torcheeg/MainThread) 😊 | Please set split_path to ./examples_pipeline/split for the next run, if you want to use the same setting for the experiment.
2024-05-16 04:53:00.115501: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 04:53:00.115614: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 04:53:00.244153: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Training: |          | 0/? [00:00<?, ?it/s]

[2024-05-16 04:53:12] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.412 train_accuracy: 0.261 

[2024-05-16 04:53:15] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.382 train_accuracy: 0.285 

[2024-05-16 04:53:18] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.366 train_accuracy: 0.309 

[2024-05-16 04:53:21] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.342 train_accuracy: 0.344 

[2024-05-16 04:53:24] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.324 train_accuracy: 0.369 

[2024-05-16 04:53:27] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.301 train_accuracy: 0.394 

[2024-05-16 04:53:31] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.279 train_accuracy: 0.425 

[2024-05-16 04:53:34] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.254 train_accuracy: 0.437 

[2024-05-16 04:53:37] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.222 train_accuracy: 0.461 

[2024-05-16 04:53:40] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.197 train_accuracy: 0.470 



Testing: |          | 0/? [00:00<?, ?it/s]

[2024-05-16 04:55:42] INFO (torcheeg/MainThread) 
[Test] test_loss: 0.928 test_accuracy: 0.625 



...

[2024-05-16 05:17:58] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.405 train_accuracy: 0.268 

[2024-05-16 05:18:01] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.373 train_accuracy: 0.309 

[2024-05-16 05:18:04] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.357 train_accuracy: 0.337 

[2024-05-16 05:18:08] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.332 train_accuracy: 0.358 

[2024-05-16 05:18:11] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.306 train_accuracy: 0.389 

[2024-05-16 05:18:14] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.279 train_accuracy: 0.417 

[2024-05-16 05:18:18] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.251 train_accuracy: 0.438 

[2024-05-16 05:18:21] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.232 train_accuracy: 0.453 

[2024-05-16 05:18:24] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.206 train_accuracy: 0.471 

[2024-05-16 05:18:28] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.187 train_accuracy: 0.492 



Testing: |          | 0/? [00:00<?, ?it/s]

[2024-05-16 05:20:44] INFO (torcheeg/MainThread) 
[Test] test_loss: 0.920 test_accuracy: 0.627 



┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6273148059844971     │
│         test_loss         │    0.9195728302001953     │
└───────────────────────────┴───────────────────────────┘

Fold 9 test accuracy:  0.6273


In [8]:
!mkdir weights

In [9]:
eeg_weights_path = './weights/eeg_weights.pt'
torch.save(model.state_dict(), eeg_weights_path)
# to load weight:
# model.load_state_dict(torch.load(eeg_weights_path))